# בניית מודל רגרסיה: הכנת ויזואליזציה של נתונים

## **רגרסיה לינארית לדלעות - שיעור 2**
#### הקדמה

עכשיו, כשיש לך את הכלים הדרושים להתחיל לבנות מודלים של למידת מכונה עם Tidymodels ו-Tidyverse, אתה מוכן להתחיל לשאול שאלות על הנתונים שלך. כשאתה עובד עם נתונים ומיישם פתרונות ML, חשוב מאוד להבין איך לשאול את השאלה הנכונה כדי למצות את הפוטנציאל של מערך הנתונים שלך.

בשיעור זה תלמד:

- איך להכין את הנתונים שלך לבניית מודלים.

- איך להשתמש ב-`ggplot2` לויזואליזציה של נתונים.

השאלה שאתה צריך תשובה עליה תקבע איזה סוג של אלגוריתמים ML תשתמש בהם. ואיכות התשובה שתקבל תהיה תלויה מאוד באופי הנתונים שלך.

בואו נראה זאת דרך תרגיל מעשי.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>יצירה מאת @allison_horst</figcaption>


## 1. ייבוא נתוני דלעות והפעלת Tidyverse

נזדקק לחבילות הבאות כדי לפרק ולהרכיב את השיעור הזה:

-   `tidyverse`: ה-[tidyverse](https://www.tidyverse.org/) הוא [אוסף של חבילות R](https://www.tidyverse.org/packages) שנועד להפוך את מדע הנתונים למהיר, קל ומהנה יותר!

ניתן להתקין אותן כך:

`install.packages(c("tidyverse"))`

הסקריפט למטה בודק אם יש לך את החבילות הנדרשות להשלמת המודול הזה ומתקין אותן עבורך במקרה שחלקן חסרות.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

עכשיו, בואו נפעיל כמה חבילות ונטעין את [הנתונים](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) שסופקו לשיעור הזה!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

הצצה מהירה באמצעות `glimpse()` מראה מיד שישנם ערכים ריקים ותערובת של מחרוזות (`chr`) ונתונים מספריים (`dbl`). ה-`Date` הוא מסוג מחרוזת ויש גם עמודה מוזרה בשם `Package` שבה הנתונים הם תערובת של `sacks`, `bins` וערכים אחרים. למעשה, הנתונים קצת מבולגנים 😤.

למעשה, זה לא מאוד נפוץ לקבל מערך נתונים שמוכן לחלוטין לשימוש ליצירת מודל ML ישר מהקופסה. אבל אל דאגה, בשיעור הזה תלמדו איך להכין מערך נתונים גולמי באמצעות ספריות R סטנדרטיות 🧑‍🔧. בנוסף, תלמדו טכניקות שונות להמחשת הנתונים.📈📊
<br>

> תזכורת: אופרטור הצינור (`%>%`) מבצע פעולות ברצף לוגי על ידי העברת אובייקט קדימה לתוך פונקציה או ביטוי קריאה. אפשר לחשוב על אופרטור הצינור כמי שאומר "ואז" בקוד שלכם.


## 2. בדיקת נתונים חסרים

אחת הבעיות הנפוצות ביותר שאנשי מדע הנתונים צריכים להתמודד איתן היא נתונים חסרים או לא שלמים. R מייצגת ערכים חסרים או לא ידועים באמצעות ערך מיוחד: `NA` (Not Available).

אז איך נוכל לדעת אם מסגרת הנתונים מכילה ערכים חסרים?
<br>
-   דרך פשוטה לבדוק זאת היא להשתמש בפונקציה הבסיסית של R בשם `anyNA`, שמחזירה אובייקטים לוגיים `TRUE` או `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

מעולה, נראה שחסרים כמה נתונים! זה מקום טוב להתחיל בו.

-   דרך נוספת תהיה להשתמש בפונקציה `is.na()` שמציינת אילו אלמנטים בעמודות ספציפיות חסרים באמצעות ערך לוגי `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

אוקיי, סיימתי את העבודה, אבל עם מסגרת נתונים גדולה כמו זו, זה יהיה לא יעיל וכמעט בלתי אפשרי לעבור על כל השורות והעמודות באופן פרטני😴.

-   דרך אינטואיטיבית יותר תהיה לחשב את סכום הערכים החסרים עבור כל עמודה:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

הרבה יותר טוב! חסרים נתונים, אבל אולי זה לא ישפיע על המשימה הנוכחית. בואו נראה מה ניתוח נוסף יביא.

> בנוסף למערכות החבילות והפונקציות המדהימות, ל-R יש תיעוד מאוד טוב. לדוגמה, השתמשו ב-`help(colSums)` או `?colSums` כדי ללמוד עוד על הפונקציה.


## 3. Dplyr: דקדוק של מניפולציה על נתונים

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>יצירה מאת @allison_horst</figcaption>


<!--![יצירה מאת \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>יצירה מאת \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), חבילה מתוך Tidyverse, היא תחביר לעיבוד נתונים שמספק סט עקבי של פעלים שעוזרים לך לפתור את האתגרים הנפוצים ביותר בעיבוד נתונים. בחלק זה, נחקור כמה מהפעלים של dplyr!  
<br>


#### dplyr::select()

`select()` היא פונקציה בחבילת `dplyr` שעוזרת לך לבחור עמודות לשמירה או להסרה.

כדי להפוך את מסגרת הנתונים שלך לקלה יותר לעבודה, הסר כמה מהעמודות שלה באמצעות `select()`, והשאר רק את העמודות שאתה צריך.

לדוגמה, בתרגיל הזה, הניתוח שלנו יכלול את העמודות `Package`, `Low Price`, `High Price` ו-`Date`. בואו נבחר את העמודות האלה.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` היא פונקציה בחבילת `dplyr` שמאפשרת ליצור או לשנות עמודות, תוך שמירה על העמודות הקיימות.

המבנה הכללי של `mutate` הוא:

`data %>%   mutate(new_column_name = what_it_contains)`

בואו נשתמש ב-`mutate` על עמודת `Date` ונבצע את הפעולות הבאות:

1.  המרת התאריכים (שכרגע הם מסוג תווים) לפורמט של חודשים (אלו תאריכים אמריקאים, כך שהפורמט הוא `MM/DD/YYYY`).

2.  חילוץ החודש מתוך התאריכים לעמודה חדשה.

ב-R, החבילה [lubridate](https://lubridate.tidyverse.org/) מקלה על עבודה עם נתוני תאריך-שעה. לכן, נשתמש ב-`dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` ונראה איך להשיג את המטרות הללו. נוכל להסיר את עמודת `Date`, מכיוון שלא נזדקק לה יותר בפעולות הבאות.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

איזה כיף! 🤩

עכשיו, בואו ניצור עמודה חדשה בשם `Price`, שמייצגת את המחיר הממוצע של דלעת. לאחר מכן, ניקח את הממוצע של העמודות `Low Price` ו-`High Price` כדי למלא את העמודה החדשה Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

כן!💪

"אבל רגע!", תגידו אחרי שתעיפו מבט על כל מערך הנתונים עם `View(pumpkins)`, "יש כאן משהו מוזר!"🤔

אם תסתכלו על העמודה `Package`, תראו שהדלעות נמכרות בהרבה תצורות שונות. חלקן נמכרות במידות של `1 1/9 bushel`, חלקן במידות של `1/2 bushel`, חלקן לפי דלעת, חלקן לפי משקל, וחלקן בארגזים גדולים ברוחבים משתנים.

בואו נאמת את זה:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

מדהים!👏

דלעות נראות כקשות מאוד לשקילה עקבית, אז בואו נסנן אותן על ידי בחירת דלעות בלבד עם המחרוזת *bushel* בעמודת `Package` ונשים זאת במסגרת נתונים חדשה בשם `new_pumpkins`.  


#### dplyr::filter() ו-stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): יוצר תת-קבוצה של הנתונים שמכילה רק **שורות** שעומדות בתנאים שלך, במקרה הזה, דלעות עם המחרוזת *bushel* בעמודת `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): מזהה את הנוכחות או ההיעדרות של תבנית בתוך מחרוזת.

חבילת [`stringr`](https://github.com/tidyverse/stringr) מספקת פונקציות פשוטות לפעולות נפוצות על מחרוזות.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

אתם יכולים לראות שהצלחנו לצמצם לכ-415 שורות נתונים המכילות דלעות לפי יחידות. 🤩  
<br>


#### dplyr::case_when()

**אבל רגע! יש עוד דבר אחד לעשות**

שמת לב שכמות הבושל משתנה בין השורות? צריך לנרמל את התמחור כך שתציג את המחיר לפי בושל, ולא לפי 1 1/9 או 1/2 בושל. הגיע הזמן לעשות קצת מתמטיקה כדי לסטנדרט את זה.

נשתמש בפונקציה [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) כדי *לשנות* את עמודת המחיר בהתאם לכמה תנאים. `case_when` מאפשרת לך לייעל מספר הצהרות `if_else()` בווקטור אחד.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

עכשיו, אנחנו יכולים לנתח את המחיר ליחידה בהתבסס על מדידת הבושל שלהם. כל המחקר הזה על בושלים של דלעות, עם זאת, מראה עד כמה `חשוב` זה `להבין את טבע הנתונים שלך`!

> ✅ לפי [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), המשקל של בושל תלוי בסוג התוצרת, מכיוון שמדובר במדידת נפח. "בושל של עגבניות, למשל, אמור לשקול 56 פאונד... עלים וירוקים תופסים יותר מקום עם פחות משקל, כך שבושל של תרד שוקל רק 20 פאונד." זה די מסובך! בואו לא נטרח לבצע המרה מבושל לפאונד, ובמקום זאת נתמחר לפי בושל. כל המחקר הזה על בושלים של דלעות, עם זאת, מראה עד כמה חשוב להבין את טבע הנתונים שלך!

> ✅ שמתם לב שדלעות שנמכרות בחצי בושל הן מאוד יקרות? האם אתם יכולים להבין למה? רמז: דלעות קטנות יקרות בהרבה מדלעות גדולות, כנראה בגלל שיש הרבה יותר מהן בכל בושל, בהתחשב במקום הלא מנוצל שתופסת דלעת פאי גדולה וחלולה אחת.


עכשיו לבסוף, רק בשביל הכיף 💁‍♀️, בואו גם נעביר את עמודת החודש למיקום הראשון, כלומר `לפני` עמודת `Package`.

משתמשים ב-`dplyr::relocate()` כדי לשנות את מיקומי העמודות.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

עבודה טובה!👌 עכשיו יש לך מערך נתונים נקי ומסודר שעליו תוכל לבנות את מודל הרגרסיה החדש שלך!  
<br>


## 4. ויזואליזציה של נתונים עם ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>אינפוגרפיקה מאת דאסאני מדיפאלי</figcaption>


<!--![Infographic by Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

יש פתגם *חכם* שאומר כך:

> "הגרף הפשוט הביא יותר מידע למוחו של מנתח הנתונים מכל כלי אחר." --- ג'ון טוקי

חלק מתפקידו של מדען הנתונים הוא להציג את האיכות והאופי של הנתונים שהוא עובד איתם. כדי לעשות זאת, הם לעיתים קרובות יוצרים ויזואליזציות מעניינות, או גרפים, תרשימים וטבלאות, שמציגים היבטים שונים של הנתונים. בדרך זו, הם יכולים להציג באופן חזותי קשרים ופערים שקשה לגלות בדרכים אחרות.

ויזואליזציות יכולות גם לעזור לקבוע את טכניקת הלמידה החישובית המתאימה ביותר לנתונים. לדוגמה, תרשים פיזור שנראה כאילו הוא עוקב אחר קו, מצביע על כך שהנתונים מתאימים לתרגיל רגרסיה ליניארית.

R מציעה מספר מערכות ליצירת גרפים, אבל [`ggplot2`](https://ggplot2.tidyverse.org/index.html) היא אחת מהמערכות האלגנטיות והגמישות ביותר. `ggplot2` מאפשרת להרכיב גרפים על ידי **שילוב רכיבים עצמאיים**.

בואו נתחיל עם תרשים פיזור פשוט עבור העמודות Price ו-Month.

במקרה הזה, נתחיל עם [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), נספק מערך נתונים ומיפוי אסתטי (עם [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) ואז נוסיף שכבות (כמו [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) עבור תרשימי פיזור.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

האם זה עלילה שימושית 🤷? האם משהו בה מפתיע אותך?

זה לא במיוחד שימושי, כי כל מה שזה עושה זה להציג את הנתונים שלך כפריסה של נקודות בחודש נתון.  
<br>


### **איך הופכים את זה לשימושי?**

כדי להציג נתונים מועילים בתרשימים, בדרך כלל צריך לקבץ את הנתונים בצורה כלשהי. לדוגמה, במקרה שלנו, מציאת המחיר הממוצע של דלעות עבור כל חודש תספק תובנות נוספות לגבי הדפוסים הבסיסיים בנתונים שלנו. זה מוביל אותנו לעוד מבט מהיר על **dplyr**:

#### `dplyr::group_by() %>% summarize()`

ניתן לחשב בקלות אגרגציה מקובצת ב-R באמצעות

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` משנה את יחידת הניתוח מהמאגר הנתונים המלא לקבוצות פרטניות, כמו לפי חודש.

-   `dplyr::summarize()` יוצר מסגרת נתונים חדשה עם עמודה אחת עבור כל משתנה קיבוץ ועמודה אחת עבור כל סטטיסטיקת סיכום שציינת.

לדוגמה, ניתן להשתמש ב-`dplyr::group_by() %>% summarize()` כדי לקבץ את הדלעות לקבוצות על בסיס עמודת **Month** ואז למצוא את **המחיר הממוצע** עבור כל חודש.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

תמציתי!✨

תכונות קטגוריות כמו חודשים מיוצגות בצורה טובה יותר באמצעות תרשים עמודות 📊. השכבות האחראיות על תרשימי עמודות הן `geom_bar()` ו-`geom_col()`. ניתן לעיין ב-`?geom_bar` כדי ללמוד עוד.

בואו נכין אחד!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 זו ויזואליזציה שימושית יותר של נתונים! נראה שהיא מצביעה על כך שהמחיר הגבוה ביותר לדלעות מתרחש בספטמבר ובאוקטובר. האם זה תואם את הציפיות שלך? למה כן או למה לא?

מזל טוב על סיום השיעור השני 👏! הכנת את הנתונים שלך לבניית מודל, ואז גילית תובנות נוספות באמצעות ויזואליזציות!



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפתו המקורית נחשב למקור הסמכותי. למידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי בני אדם. איננו נושאים באחריות לכל אי-הבנה או פרשנות שגויה הנובעת משימוש בתרגום זה.
